In [2]:
import torch
from torch import hub

In [3]:
import torchvision

In [4]:
img_t = torch.randn(3, 5, 5)
weights = torch.tensor([0.2126, 0.7152, 0.0722])

In [5]:
batch_t = torch.randn(2, 3, 5, 5)

In [6]:
"""
バッチ次元の有無により
RGBのチャネルは0か1のため
終わりから数えると-3にRGBがくる
"""

'\nバッチ次元の有無により\nRGBのチャネルは0か1のため\n終わりから数えると-3にRGBがくる\n'

In [7]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)

img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [8]:
#重みを1次元から3次元へ[3,1,1]
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze(-1)
#broadcastingされてimgには「3、5、5」としてbacthには「2,3,5,5」となり次元が追加される
img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)

img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
print(weights.shape)
print(unsqueezed_weights.shape, img_t.shape ,img_weights.shape, img_gray_weighted.shape) 
print(unsqueezed_weights.shape, batch_t.shape, batch_weights.shape, batch_gray_weighted.shape) 

torch.Size([3])
torch.Size([3, 1, 1]) torch.Size([3, 5, 5]) torch.Size([3, 5, 5]) torch.Size([5, 5])
torch.Size([3, 1, 1]) torch.Size([2, 3, 5, 5]) torch.Size([2, 3, 5, 5]) torch.Size([2, 5, 5])


In [9]:
img_t[1][1],img_weights[1][1],unsqueezed_weights[1]

(tensor([-0.0727, -1.6070,  0.8864, -0.5384,  0.8892]),
 tensor([-0.0520, -1.1493,  0.6340, -0.3851,  0.6360]),
 tensor([[0.7152]]))

In [10]:
#数値型
#必要に応じて低ビットを使う

In [11]:
double_points = torch.ones(10, 2 ,dtype=torch.double)#64倍制度浮動小数点
short_points = torch.tensor([[1,2], [3,4]], dtype=torch.short)#short=16符号付き整数
bit_points = torch.zeros(3,3, dtype=torch.int)

In [12]:
double_points.dtype

torch.float64

In [13]:
short_points.dtype

torch.int16

In [14]:
bit_points.dtype

torch.int32

In [15]:
#キャスト機能も使える
moto=torch.zeros(10, 2)
henka=moto.short()
moto.dtype, henka.dtype 

(torch.float32, torch.int16)

In [16]:
#関数でもいける
anather_use = henka.to(torch.double)
anather_use.dtype

torch.float64

In [17]:
#テンソルの演算
#テンソルの演算はほとんどtorchモジュールで可能
#http://pytorch.org/docs

In [18]:
a = torch.ones(3, 2)
a_t = torch.transpose(a, 0, 1)
#次元の交換
print(a.shape, a_t.shape)

torch.Size([3, 2]) torch.Size([2, 3])


In [19]:
a_t = a.transpose(0,1)
print(a_t.shape)

torch.Size([2, 3])


In [20]:
#テンソルとストレージの表示関係

In [21]:
big_points=torch.zeros(6,4,2)
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_points = points[1]

In [22]:
second_points

tensor([5., 3.])

In [23]:
print(points.size())
print(second_points.size(), second_points.shape)
print(big_points.shape)

torch.Size([3, 2])
torch.Size([2]) torch.Size([2])
torch.Size([6, 4, 2])


# オフセット

In [35]:
#横に並べて、最初の要素からいくつ目か
#points[2]は二つの要素の点をスキップしないといけない
#pointsはそれ自身しかない
print(points[1].storage_offset(), points[2].storage_offset(), points.storage_offset(),)
print(big_points.storage_offset(), big_points[1,1,0].storage_offset())

2 4 0
0 10


In [33]:
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

# ストライド

In [28]:
#各インデックスを増加した時スキップしないといけない要素数
#各次元の要素数ー1
points.shape,points.stride(), second_points.stride(), 

(torch.Size([3, 2]), (2, 1), (1,))

In [30]:
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [27]:
big_points.stride()

(8, 2, 1)

In [121]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_points = points[1]
second_points

tensor([5., 3.])

In [122]:
second_points[1]=10.0

In [123]:
points

tensor([[ 4.,  1.],
        [ 5., 10.],
        [ 2.,  1.]])

In [29]:
x = torch.tensor([[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]])
x.stride()

(5, 1)

In [124]:
#サブテンソルの変更は元のテンソルへも影響がある
#影響を避けるにはclone()

second_points = points[1].clone()
second_points

tensor([ 5., 10.])

In [125]:
second_points[1] =200
points

tensor([[ 4.,  1.],
        [ 5., 10.],
        [ 2.,  1.]])

In [127]:
#天地
points, points.t()

(tensor([[ 4.,  1.],
         [ 5., 10.],
         [ 2.,  1.]]),
 tensor([[ 4.,  5.,  2.],
         [ 1., 10.,  1.]]))

In [128]:
#高次元での天地は二次元を指定する

some_t = torch.ones(3,4,5)
transpose_some_t = some_t.transpose(0,2)
print(some_t.shape, transpose_some_t.shape)

torch.Size([3, 4, 5]) torch.Size([5, 4, 3])


In [133]:
#要素が連続しているか確認
points.shape, points.is_contiguous()

(torch.Size([3, 2]), True)

In [137]:
#天地した行列は連続していない
points.t().shape, points.t().is_contiguous()

(torch.Size([2, 3]), False)

In [138]:
points_t = points.t()
points_t_cont = points_t.contiguous()

In [139]:
#contiguous()で新しいテンソルが作られる
points_t_cont.is_contiguous()

True

In [ ]:
#GPUへの転送
#deviceという属性にはテンソルの場所を示す
points_gpu = torch.tensor([[4.0, 1.0],[5.0, 3.0], [2.0, 1.0], device='cuda')

In [ ]:
#cpuで作成したテンソルの送信
points_gpu = points.to(device='cuda')

In [ ]:
#複数のGPUがある場合は整数で選択
points_gpu = points.to(device='cuda:0')

In [3]:
#numpyとの相互運用性

In [1]:
import torch
points = torch.ones(3, 4)
points_np = points.numpy()
points_np, points

(array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32),
 tensor([[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]))

In [2]:
points_np[1][1]=3

In [9]:
#メモリを共有している
points_np, points

(array([[1., 1., 1., 1.],
        [1., 3., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32),
 tensor([[1., 1., 1., 1.],
         [1., 3., 1., 1.],
         [1., 1., 1., 1.]]))

In [10]:
#テンソルの保存とロード

In [17]:
#ファイル名で保存
torch.save(points, "ourpoints.t")

In [18]:
#ファイル変数として保存
with open ('ourpoints2.t', "wb") as f:
    torch.save(points, f)

In [4]:
#ロード
points_load = torch.load('ourpoints.t')
print(points_load)

tensor([[1., 1., 1., 1.],
        [1., 3., 1., 1.],
        [1., 1., 1., 1.]])


In [26]:
with open('ourpoints2.t','rb') as f:
    points = torch.load(f)

In [3]:
#h5pyによるHDF5での保存
#HDF5形式はkey-value辞書形式で構成された多次元配列を表現するためのフォーマット
import h5py

In [4]:
f = h5py.File('ourpoints.hdf5', 'w')
dset = f.create_dataset('coords', data=points.numpy())
f.close()

In [6]:
f = h5py.File('ourpoints.hdf5', "r")
dset = f['coords']
#関心のある要素のみメモリに載せることができる
last_points = dset[-2:]
print(dset)
print(last_points)

<HDF5 dataset "coords": shape (3, 4), type "<f4">
[[1. 3. 1. 1.]
 [1. 1. 1. 1.]]


In [7]:
#演習問題

In [28]:
a = list(range(9))
print(a)
a= torch.tensor(a)
print(a)
print(a.size(), a.storage_offset(), a.stride())

[0, 1, 2, 3, 4, 5, 6, 7, 8]
tensor([0, 1, 2, 3, 4, 5, 6, 7, 8])
torch.Size([9]) 0 (1,)


In [29]:
b = a.view(3, 3)
b

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])

In [30]:
b[1][2] = 100
a,b

(tensor([  0,   1,   2,   3,   4, 100,   6,   7,   8]),
 tensor([[  0,   1,   2],
         [  3,   4, 100],
         [  6,   7,   8]]))

In [31]:
c = b[1:,1:]
print(c)
print(c.size(), c.storage_offset(), c.stride())

tensor([[  4, 100],
        [  7,   8]])
torch.Size([2, 2]) 4 (3, 1)


In [33]:
print(torch.cos(a))

tensor([ 1.0000,  0.5403, -0.4161, -0.9900, -0.6536,  0.8623,  0.9602,  0.7539,
        -0.1455])


In [34]:
print(torch.rsqrt(a))

tensor([   inf, 1.0000, 0.7071, 0.5774, 0.5000, 0.1000, 0.4082, 0.3780, 0.3536])


In [35]:
a

tensor([  0,   1,   2,   3,   4, 100,   6,   7,   8])